# ZeroxPDFLoader

This sample provides a quick overview for getting started with `ZeroxPDFLoader` [document loader](https://python.langchain.com/docs/concepts/document_loaders). For detailed documentation of all ZeroxPDFLoader features and configurations head to the [API reference](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.ZeroxPDFLoader.html).

  

## Overview
`ZeroxPDFLoader` is a document loader that leverages the [Zerox](https://github.com/getomni-ai/zerox) library. Zerox converts PDF documents into images, processes them using a vision-capable language model, and generates a structured Markdown representation. This loader allows for asynchronous operations and provides page-level document extraction.

### Integration details

| Class | Package | Local | Serializable | JS support|
| :--- | :--- | :---: | :---: |  :---: |
| [PyMuPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.ZeroxPDFLoader.html) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) | ✅ | ❌ | ❌ |

---------   

### Loader features

| Source | Document Lazy Loading | Native Async Support | Extract Images | Extract Tables |
| :---: | :---: | :---: | :---: |:---: |
| ZeroxPDFLoader | ✅ | ❌ | ✅ | ✅ |

  

## Setup

`ZeroxPDFLoader` enables PDF text extraction using vision-capable language models by converting each page into an image and processing it asynchronously. To use this loader, you need to specify a model and configure any necessary environment variables for Zerox, such as API keys.

If you're working in an environment like Jupyter Notebook, you may need to handle asynchronous code by using `nest_asyncio`. You can set this up as follows:

```python
import nest_asyncio
nest_asyncio.apply()
```

In [1]:
import os
from getpass import getpass

# use nest_asyncio (only necessary inside of jupyter notebook)
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key =")

nest_asyncio.apply()

### Credentials
Appropriate credentials need to be set up in environment variables. The loader supports number of different models and model providers. See _Usage_ header below to see few examples or [Zerox documentation](https://github.com/getomni-ai/zerox) for a full list of supported models.


In [2]:
from getpass import getpass

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key =")

nest_asyncio.apply()


If you want to get automated best in-class tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [3]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

Install **langchain_community** and **py-zerox**.

In [4]:
%pip install -qU langchain_community py-zerox

Note: you may need to restart the kernel to use updated packages.


## Initialization

Now we can instantiate our model object and load documents:

In [5]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [6]:
def run_in_thread():
    loop = asyncio.get_event_loop()  # Créer une nouvelle boucle d'événements pour ce thread
    result = loop.run_until_complete(toto())
    return result

from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=1)
async_result = pool.apply_async(run_in_thread) # tuple of args for foo
result=async_result.get()
print(result)


NameError: name 'toto' is not defined

In [26]:
loop = asyncio.get_running_loop()

def _run_in_thread(loop):
    loop = asyncio.get_event_loop()  # Créer une nouvelle boucle d'événements pour ce thread
    result = loop.run_until_complete(toto())
    return result

from multiprocessing.pool import ThreadPool
pool = ThreadPool(processes=1)
async_result = pool.apply_async(_run_in_thread,(loop,))  # tuple of args for foo
result = async_result.get()
print(result)

toto
hello


In [7]:
from langchain_community.document_loaders.pdf import ZeroxPDFLoader

file_path = "./example_data/layout-parser-paper.pdf"
loader = ZeroxPDFLoader(file_path)

## Load

In [8]:
docs = loader.load()
docs[0]

/home/pprados/workspace.bda/langchain/libs/community/.venv/lib/python3.12/site-packages/pyzerox/models/modellitellm.py:52: UserWarning: 
    Custom system prompt was provided which overrides the default system prompt. We assume that you know what you are doing.  
    . Default prompt for zerox is:
 
    Convert the following PDF page to markdown.
    Return only the markdown with no explanation text.
    Do not exclude any content from the page.
    
  warnings.warn(f"{Messages.CUSTOM_SYSTEM_PROMPT_WARNING}. Default prompt for zerox is:\n {DEFAULT_SYSTEM_PROMPT}")


Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-06-22T01:27:10+00:00', 'author': '', 'keywords': '', 'moddate': '2021-06-22T01:27:10+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': 'False', 'total_pages': 16, 'source': './example_data/layout-parser-paper.pdf', 'num_pages': 16, 'page': 0}, page_content='# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis\n\nZejian Shen¹, Ruochen Zhang², Melissa Dell³, Benjamin Charles Germain Lee⁴, Jacob Carlson³, and Weining Li⁵\n\n¹ Allen Institute for AI  \nshannons@allenai.org  \n² Brown University  \nruochen_zhang@brown.edu  \n³ Harvard University  \n{melissadell, jacob.carlson}@fas.harvard.edu  \n⁴ University of Washington  \nbgcl@cs.washington.edu  \n⁵ University of Waterloo  \nw4221@uwaterloo.ca  \n\n**Abstract.** Recent advances in document image analysi

In [9]:
import pprint

pprint.pp(docs[0].metadata)

{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf',
 'num_pages': 16,
 'page': 0}


## Lazy Load


In [ ]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # do some paged operation, e.g.
        # index.upsert(page)

        pages = []
len(pages)

/home/pprados/workspace.bda/langchain/libs/community/.venv/lib/python3.12/site-packages/pyzerox/models/modellitellm.py:52: UserWarning: 
    Custom system prompt was provided which overrides the default system prompt. We assume that you know what you are doing.  
    . Default prompt for zerox is:
 
    Convert the following PDF page to markdown.
    Return only the markdown with no explanation text.
    Do not exclude any content from the page.
    
  warnings.warn(f"{Messages.CUSTOM_SYSTEM_PROMPT_WARNING}. Default prompt for zerox is:\n {DEFAULT_SYSTEM_PROMPT}")


In [9]:
print(pages[0].page_content[:100])
pprint.pp(pages[0].metadata)

5.1 A Comprehensive Historical Document Digitization Pipeline

The digitization of historical docume
{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf',
 'num_pages': 16,
 'page': 10}


The metadata attribute contains at least the following keys:
- source
- page (if in mode *page*)
- total_page
- creationdate
- creator
- producer

Additional metadata are specific to each parser.
These pieces of information can be helpful (to categorize your PDFs for example).

## Splitting mode & custom pages delimiter

When loading the PDF file you can split it in two different ways:
- By page
- As a single text flow

By default PDFPlumberLoader will split the PDF by page.

### Extract the PDF by page. Each page is extracted as a langchain Document object:

In [10]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
)
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

16
{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf',
 'num_pages': 16,
 'page': 0}


In this mode the pdf is split by pages and the resulting Documents metadata contains the page number. But in some cases we could want to process the pdf as a single text flow (so we don't cut some paragraphs in half). In this case you can use the *single* mode :

### Extract the whole PDF as a single langchain Document object:

In [11]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
)
docs = loader.load()
print(len(docs))
pprint.pp(docs[0].metadata)

1
{'producer': 'pdfTeX-1.40.21',
 'creator': 'LaTeX with hyperref',
 'creationdate': '2021-06-22T01:27:10+00:00',
 'author': '',
 'keywords': '',
 'moddate': '2021-06-22T01:27:10+00:00',
 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live '
                    '2020) kpathsea version 6.3.2',
 'subject': '',
 'title': '',
 'trapped': 'False',
 'total_pages': 16,
 'source': './example_data/layout-parser-paper.pdf',
 'num_pages': 16}


Logically, in this mode, the ‘page_number’ metadata disappears. Here's how to clearly identify where pages end in the text flow :

### Add a custom *pages_delimiter* to identify where are ends of pages in *single* mode:

In [12]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="single",
    pages_delimiter="\n-------THIS IS A CUSTOM END OF PAGE-------\n",
)
docs = loader.load()
print(docs[0].page_content[:5780])

LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis

Zejian Shen¹ (✉), Ruochen Zhang², Melissa Dell³, Benjamin Charles Germain Lee⁴, Jacob Carlson³, and Weining Li⁵

¹ Allen Institute for AI
shannons@allenai.org  
² Brown University  
ruochen_zhang@brown.edu  
³ Harvard University  
{melissadell, jacob.carlson}@fas.harvard.edu  
⁴ University of Washington  
bgcl@cs.washington.edu  
⁵ University of Waterloo  
w422ii@uwaterloo.ca  

Abstract. Recent advances in document image analysis (DIA) have been primarily driven by the application of neural networks. Ideally, research outcomes could be easily deployed in production and extended for further investigation. However, various factors like loosely organized codebases and sophisticated model configurations complicate the easy reuse of important innovations by a wide audience. Though there have been on-going efforts to improve reusability and simplify deep learning (DL) model development in disciplines like natur

This could simply be \n, or \f to clearly indicate a page change, or \<!-- PAGE BREAK --> for seamless injection in a Markdown viewer without a visual effect.

# Extract images from the PDF

You can extract images from your PDFs with a choice of three different solutions:
- rapidOCR (lightweight Optical Character Recognition tool)
- Tesseract (OCR tool with high precision)
- Multimodal language model

Because Zerox uses a multimodal LLM, it is responsible for OCR analysis. Nevertheless, to maintain compatibility with other parsers, we simulate image_parsers where possible.

You can tune these functions to choose the output format of the extracted images among *html*, *markdown* or *text*

The result is inserted between the last and the second-to-last paragraphs of text of the page.

In [13]:
%pip install -qU rapidocr-onnxruntime

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_community.document_loaders.parsers import RapidOCRBlobParser

loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    images_inner_format="markdown-img",
    images_parser=RapidOCRBlobParser(),
)
docs = loader.load()

print(docs[5].page_content)

6 Z. Shen et al.

![The relationship between the three types of layout data structures. Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.](image-placeholder)

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such a model is not readily available, LayoutParser also supports training customized layout models and community sharing of the models (detailed in Section 3.5).

3.2 Layout Data Structures

A critical feature of LayoutParser is the implementation of a series of data 

With this simulation, RapidOCR is not invoked.

### Extract images from the PDF with Tesseract:

In [15]:
%pip install -qU pytesseract

Note: you may need to restart the kernel to use updated packages.


In [16]:
from langchain_community.document_loaders.parsers import TesseractBlobParser

loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    images_inner_format="html-img",
    images_parser=TesseractBlobParser(),
)
docs = loader.load()
print(docs[5].page_content)

6 Z. Shen et al.

![The relationship between the three types of layout data structures. Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.](image_link)

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such a model is not readily available, LayoutParser also supports training customized layout models and community sharing of the models (detailed in Section 3.5).

## 3.2 Layout Data Structures

A critical feature of LayoutParser is the implementation of a series of data stru

With this simulation, Tesseract is not invoked.

### Extract images from the PDF with multimodal model:

In [17]:
%pip install -qU langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [18]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [19]:
from getpass import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key =")

In [20]:
from langchain_community.document_loaders.parsers import LLMImageBlobParser
from langchain_openai import ChatOpenAI

loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    images_inner_format="markdown-img",
    images_parser=LLMImageBlobParser(model=ChatOpenAI(model="gpt-4o", max_tokens=1024)),
)
docs = loader.load()
print(docs[5].page_content)

![Fig. 2: The relationship between the three types of layout data structures.](#)

Coordinate supports three kinds of variation; TextBlock consists of the coordinate information and extra features like block text, types, and reading orders; a Layout object is a list of all possible layout elements, including other Layout objects. They all support the same set of transformation and operation APIs for maximum flexibility.

Shown in Table 1, LayoutParser currently hosts 9 pre-trained models trained on 5 different datasets. Description of the training dataset is provided alongside with the trained models such that users can quickly identify the most suitable models for their tasks. Additionally, when such a model is not readily available, LayoutParser also supports training customized layout models and community sharing of the models (detailed in Section 3.5).

### 3.2 Layout Data Structures

A critical feature of LayoutParser is the implementation of a series of data structures and operat

With this simulation, the format is injected into the prompt.

# Extract tables from the PDF

With PyMUPDF you can extract tables from your PDFs in *html*, *markdown* or *csv* format :

In [21]:
loader = ZeroxPDFLoader(
    "./example_data/layout-parser-paper.pdf",
    mode="page",
    extract_tables="markdown",
)
docs = loader.load()
print(docs[4].page_content)

| Dataset      | Base Model | Large Model | Notes                                                  |
|--------------|------------|-------------|--------------------------------------------------------|
| PubLayNet    | F / M      | M           | Layouts of modern scientific documents                  |
| PubLayNet    | M          | M           | Layouts of scanned modern magazines and scientific reports |
| Newspaper    | F          | -           | Layouts of scanned US newspapers from the 20th century  |
| TableBank    | F          | F           | Table region on modern scientific and business documents |
| HLDataset    | F / M      | -           | Layouts of history Japanese documents                    |

1. For each dataset, we train several models of different sizes for different needs (the trade-off between accuracy vs. computational cost). For "base model" and "large model", we refer to using the ResNet 50 or ResNet 101 backbones. One can train models of different architectures,

## Working with Files

Many document loaders involve parsing files. The difference between such loaders usually stems from how the file is parsed, rather than how the file is loaded. For example, you can use `open` to read the binary content of either a PDF or a markdown file, but you need different parsing logic to convert that binary data into text.

As a result, it can be helpful to decouple the parsing logic from the loading logic, which makes it easier to re-use a given parser regardless of how the data was loaded.
You can use this strategy to analyze different files, with the same parsing parameters.

In [22]:
from langchain_community.document_loaders import FileSystemBlobLoader
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import ZeroxPDFParser

loader = GenericLoader(
    blob_loader=FileSystemBlobLoader(
        path="./example_data/",
        glob="*.pdf",
    ),
    blob_parser=ZeroxPDFParser(),
)
docs = loader.load()
print(docs[0].page_content)
pprint.pp(docs[0].metadata)

# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis

Zejian Shen¹, Ruochen Zhang², Melissa Dell³, Benjamin Charles Germain Lee⁴, Jacob Carlson³, and Weining Li⁵

¹ Allen Institute for AI  
shannon@allenai.org  
² Brown University  
ruochen_zhang@brown.edu  
³ Harvard University  
{melissad1, jacob.carlson}@fas.harvard.edu  
⁴ University of Washington  
bgcl@cs.washington.edu  
⁵ University of Waterloo  
w422ii@uwaterloo.ca  

## Abstract

Recent advances in document image analysis (DIA) have been primarily driven by the application of neural networks. Ideally, research outcomes could be easily deployed in production and extended for further investigation. However, various factors like loosely organized codebases and sophisticated model configurations complicate the easy reuse of important innovations by a wide audience. Though there have been on-going efforts to improve reusability and simplify deep learning (DL) model development in disciplines like natur

It is possible to work with files from cloud storage.

In [ ]:
from langchain_community.document_loaders import CloudBlobLoader
from langchain_community.document_loaders.generic import GenericLoader

loader = GenericLoader(
    blob_loader=CloudBlobLoader(
        url="s3:/mybucket",  # Supports s3://, az://, gs://, file:// schemes.
        glob="*.pdf",
    ),
    blob_parser=ZeroxPDFParser(),
)
docs = loader.load()
print(docs[0].page_content)
pprint.pp(docs[0].metadata)

## API reference

### `ZeroxPDFLoader`

This loader class initializes with a file path and model type, and supports custom configurations via `zerox_kwargs` for handling Zerox-specific parameters.

**Arguments**:
- `file_path` (Union[str, Path]): Path to the PDF file.
- `model` (str): Vision-capable model to use for processing in format `<provider>/<model>`.
Some examples of valid values are:
  - `model = "gpt-4o-mini" ## openai model`
  - `model = "azure/gpt-4o-mini"`
  - `model = "gemini/gpt-4o-mini"`
  - `model="claude-3-opus-20240229"`
  - `model = "vertex_ai/gemini-1.5-flash-001"`
  - See more details in [Zerox documentation](https://github.com/getomni-ai/zerox)
  - Defaults to `"gpt-4o-mini".`
- `**zerox_kwargs` (dict): Additional Zerox-specific parameters such as API key, endpoint, etc.
  - See [Zerox documentation](https://github.com/getomni-ai/zerox)

**Methods**:
- `lazy_load`: Generates an iterator of `Document` instances, each representing a page of the PDF, along with metadata including page number and source.

See full API documentaton [here](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.ZeroxPDFLoader.html)